# Fishery dataset: Predicting weight with machine learning

In this project, we aim to create machine learning models to predict the live weight of caught of fish from a fishing operation. We will try to uncover important and predictive features of the fishing operations which will be used as parameters in our models. 

Our goal is to use create three supervised learning models which all predict the same outcome.

Our methodology is divided into three parts: 

**[Part A - Preprocessing](#pre_processing)**

This part starts by initializing python and all the libraries we use to process the data. Further we perform exploratory data analysis (EDA) to understand the features, remove anomalies in the dataset and perform normalizations.

**Part b - Supervised learning**

**Part c - Unsupervised learning**

<a id="pre_processing"></a>
## Part A - Preprocessing

In this part we will analyse and prepare the data for our machine learning models.

### Step 0 - Importing libraries and reading the data

To start the project, we need to import the most common libraries we will use to explore the data. This includes pandas, numpy, matplotlib, seaborn, and sklearn.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
plt.style.use('ggplot')

### Step 1 - Reading the data

The data from the fishery reports are in the form of a CSV file. We read this file into a Pandas dataframe allowing for easy analysis and manipulation.

To ensure that the data was sucessfully loaded into the DataFrame, it can be useful to inspect the initial content. To do so we can use the `df.head(n)` function to display the first n rows and the column headers of the dataframe. 

*NOTE:* The filepath assumes that the csv file is located in a "Resources" directory in the project root directory. Please ensure that this directory and file is included prior to running the code. 

In [3]:
df = pd.read_csv('../Resources/elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv', delimiter=';')
df.head(5)

,Melding ID,Meldingstidspunkt,Meldingsdato,Meldingsklokkeslett,Starttidspunkt,Startdato,Startklokkeslett,Startposisjon bredde,Startposisjon lengde,Hovedområde start (kode),...,Art - FDIR,Art - gruppe (kode),Art - gruppe,Rundvekt,Lengdegruppe (kode),Lengdegruppe,Bruttotonnasje 1969,Bruttotonnasje annen,Bredde,Fartøylengde
0,1497177,01.01.2018,01.01.2018,00:00,31.12.2017,31.12.2017,00:00,"-60,35","-46,133",NaN,...,Antarktisk krill,506.0,Antarktisk krill,706714.0,5.0,28 m og over,9432.0,NaN,"19,87","133,88"
1,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Hyse,202.0,Hyse,9594.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
2,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Torsk,201.0,Torsk,8510.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
3,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Blåkveite,301.0,Blåkveite,196.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"
4,1497178,01.01.2018,01.01.2018,00:00,30.12.2017 23:21,30.12.2017,23:21,"74,885","16,048",20.0,...,Sei,203.0,Sei,134.0,5.0,28 m og over,1476.0,NaN,"12,6","56,8"


### Step 2 - Understanding the data

Now that we've loaded and validated the data, we can start our analysis. The initial is to familiarize ourselves with the data to understand what we're working with.

In [13]:
shape = df.shape
print("DataFrame contents")
print(f"Rows: {shape[0]}, Columns: {shape[1]}")

DataFrame contents
Rows: 305434, Columns: 45


In [27]:
print("DataFrame column names and their datatypes")
df.dtypes

DataFrame column names and their datatypes


Melding ID                    int64
Meldingstidspunkt            object
Meldingsdato                 object
Meldingsklokkeslett          object
Starttidspunkt               object
Startdato                    object
Startklokkeslett             object
Startposisjon bredde         object
Startposisjon lengde         object
Hovedområde start (kode)    float64
Hovedområde start            object
Lokasjon start (kode)       float64
Havdybde start                int64
Stopptidspunkt               object
Stoppdato                    object
Stoppklokkeslett             object
Varighet                      int64
Fangstår                      int64
Stopposisjon bredde          object
Stopposisjon lengde          object
Hovedområde stopp (kode)    float64
Hovedområde stopp            object
Lokasjon stopp (kode)       float64
Havdybde stopp                int64
Trekkavstand                float64
Redskap FAO (kode)           object
Redskap FAO                  object
Redskap FDIR (kode)         

In [26]:
df.describe()

,Melding ID,Hovedområde start (kode),Lokasjon start (kode),Havdybde start,Varighet,Fangstår,Hovedområde stopp (kode),Lokasjon stopp (kode),Havdybde stopp,Trekkavstand,Redskap FDIR (kode),Hovedart - FDIR (kode),Art - FDIR (kode),Art - gruppe (kode),Rundvekt,Lengdegruppe (kode),Bruttotonnasje 1969,Bruttotonnasje annen
count,3.054340e+05,303433.000000,303433.000000,305434.000000,305434.000000,305434.000000,303472.000000,303472.000000,305434.000000,3.054100e+05,305246.000000,300456.000000,300452.000000,300452.000000,3.004560e+05,304750.000000,234005.000000,74774.000000
mean,1.658783e+06,14.463737,19.074712,-228.025292,537.095526,2017.999941,14.430415,18.883353,-229.084850,1.566397e+04,46.489746,1326.729934,1414.625914,259.746585,7.438208e+03,4.575032,1408.386975,186.172573
std,9.130738e+04,13.001244,18.469340,226.062493,2201.624688,0.007677,12.973150,18.361244,224.277365,9.033085e+04,13.534202,614.506560,633.188386,320.124913,4.281086e+04,0.692769,1148.384145,165.761157
min,1.497177e+06,0.000000,0.000000,-5388.000000,0.000000,2017.000000,0.000000,0.000000,-5388.000000,0.000000e+00,11.000000,412.000000,211.000000,101.000000,0.000000e+00,3.000000,104.000000,21.000000
25%,1.567228e+06,5.000000,7.000000,-273.000000,123.000000,2018.000000,5.000000,7.000000,-274.000000,2.533000e+03,32.000000,1022.000000,1022.000000,201.000000,6.400000e+01,4.000000,496.000000,87.000000
50%,1.674230e+06,8.000000,12.000000,-196.000000,296.000000,2018.000000,8.000000,12.000000,-198.000000,7.598000e+03,51.000000,1032.000000,1032.000000,203.000000,3.000000e+02,5.000000,1184.000000,149.000000
75%,1.735590e+06,20.000000,24.000000,-128.000000,494.000000,2018.000000,20.000000,24.000000,-127.000000,2.259900e+04,55.000000,1038.000000,2202.000000,302.000000,2.236000e+03,5.000000,2053.000000,236.000000
max,1.800291e+06,81.000000,87.000000,1220.000000,125534.000000,2018.000000,81.000000,87.000000,1616.000000,1.588863e+07,80.000000,6619.000000,6619.000000,9903.000000,1.100000e+06,5.000000,9432.000000,1147.000000


The dataset contains in total 305.434 rows of data, with 45 columns. Having a large dataset is beneficial for our machine learning purposes. Generally having more data gives us flexibility in choosing relevant features. It also grants us a larger training, testing, and validation datasets when building our models. This can result in higher accuracy, granted the data is well-prepared and have good quality.

# Step 2 - Data preparation

From the previous step we saw that data is 

Columns such as "Bredde" and "Fartøylengde" is intereperet as object, but they should be float

The dataset contains several columns which are unnecessary for our purpose. This includes the message report metadata, as well as duplicate columns which represents the same information in codes instead of words.

Let's remove these columns.

In [ ]:
df = df[[
    'Starttidspunkt','Startposisjon bredde', 'Startposisjon lengde', 'Hovedområde start',
    'Havdybde start', 'Stopptidspunkt', 'Varighet', 'Fangstår', 'Stopposisjon bredde', 
    'Stopposisjon lengde', 'Hovedområde stopp', 'Havdybde stopp', 'Trekkavstand', 'Redskap FDIR',
    'Art - FDIR', 'Art - gruppe', 'Rundvekt', 'Bruttotonnasje annen', 
    'Bruttotonnasje 1969', 'Bredde', 'Fartøylengde'
    ]].copy()

As we noticed above, several of the datatypes which are supposed to be continuous numeric values but are interpreted as objects. This is likely because the numbers are all using comma as decimal seperator, which is the standard representation in Norway. There are also several date and time object columns which needs to be converted to datetime.

In [ ]:
df['Starttidspunkt'] = pd.to_datetime(df['Starttidspunkt'], format='mixed')
df['Stopptidspunkt'] = pd.to_datetime(df['Stopptidspunkt'], format='mixed')

df['Fartøylengde'] = pd.to_numeric(df['Fartøylengde'].str.replace(',', '.'))
df['Bredde'] = pd.to_numeric(df['Bredde'].str.replace(',', '.'))

df['Startposisjon bredde'] = pd.to_numeric(df['Startposisjon bredde'].str.replace(',', '.'))
df['Startposisjon lengde'] = pd.to_numeric(df['Startposisjon lengde'].str.replace(',', '.'))

### Null Values 
Next, we'll explore null values

In [ ]:
df.isna().sum()

Notable observations from this is the "Bruttotonnasje" columns.

In [ ]:
# Filtrer DataFrame for å få rader hvor begge kolonnene ikke er null
filtrert_df = df.query("`Bruttotonnasje annen`.notnull() and `Bruttotonnasje 1969`.notnull()")
filtrert_df = filtrert_df[['Bruttotonnasje annen', 'Bruttotonnasje 1969']]
filtrert_df

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(filtrert_df['Bruttotonnasje annen'], filtrert_df['Bruttotonnasje 1969'], color='blue', alpha=0.01)
plt.title('Bruttotonnasje annen vs 1969')
plt.xlabel('Bruttotonnasje annen')
plt.ylabel('Bruttotonnasje 1969')
plt.show()

In [ ]:
df.drop(['Bruttotonnasje 1969', 'Bruttotonnasje annen'], axis=1, inplace=True)

In [ ]:
df = df.dropna()

## Step 3 - Feature Understanding

Here we will primarily use univariate analysis to understand the individual variables better.

To start off it's useful to draw histograms of all numeric variables. This will help us understand the variance of the data, help us identify outliers which can negatively affect our models, etc.

In [ ]:
df.hist(bins=60, figsize=(20,15));plt.show()

In the resulting histogram its clear that some columns, such as "Varighet" and "Rundvekt" have outliers that can affect the quality of our models. To further analyse these variables a boxplot is useful.  

In [ ]:
def draw_boxplots(variables):
    num_of_features = len(variables)
    
    num_of_columns = 5
    num_of_rows = int(np.ceil(num_of_features / num_of_columns))
    
    fig, axes = plt.subplots(nrows=num_of_rows, ncols=num_of_columns, figsize=(15, num_of_rows * num_of_columns))
    axes = axes.flatten()

    for i, col in enumerate(variables):
        df.boxplot(column=col, ax=axes[i])
    
    # Hide empty subplots when (number_of_features < num_of_rows * num_of_columns)
    for i in range(num_of_features, len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

In [ ]:
heavy_outliers = ['Rundvekt', 'Varighet', 'Trekkavstand', 'Havdybde start']
draw_boxplots(heavy_outliers)

Lets count the numer of outliers

In [ ]:
def find_outlier_value(feature : str):
    """
    Outlier is defined as 1.5 * IQR +- nearest quartile
    
    :param feature: name of the column in the DataFrame  
    :return: tuple (lower_bound, upper_bound) which represents the upper and lower bound of outliers in the DataFrame column. 
    """
    q1 = df[feature].quantile(0.25)
    q2 = df[feature].quantile(0.75)
    iqr = q2 - q1
    
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q2 + 1.5 * iqr
    lower_bound_extreme = q1 - 3 * iqr
    upper_bound_extreme = q2 + 3 * iqr
    
    return lower_bound, upper_bound, lower_bound_extreme, upper_bound_extreme

In [ ]:
shape = []

for feature in heavy_outliers:
    lower, upper, lower_extreme, upper_extreme = find_outlier_value(feature)
    count = df.query(f'(`{feature}` < @lower) or (`{feature}` > @upper)').shape[0]
    count_extreme = df.query(f'(`{feature}` < @lower_extreme) or (`{feature}` > @upper_extreme)').shape[0]
    shape.append({'Feature': feature, 'Outliers': count, 'Extreme outliers': count_extreme})

outliers_df = pd.DataFrame(shape)
print(outliers_df)

We have many (extreme) outliers, specificly in the feature "Rundvekt". We need to deal with this.

Lets explore some normalization options.

In [ ]:
_,_,_,extreme_upper = find_outlier_value('Rundvekt')

df['Rundvekt_clipped'] = df['Rundvekt'].clip(0, extreme_upper)

In [ ]:
df['Rundvekt_clipped'].hist(bins=50)

In [ ]:
df['Rundvekt_log'] = np.log(df['Rundvekt'] + 1)
df['Varighet_log'] = np.log(df['Varighet'] + 1)
df['Trekkavstand_log'] = np.log(df['Trekkavstand'] + 1)

df['Rundvekt_log'].hist(bins=50)
df['Varighet_log'].hist(bins=50)
df['Trekkavstand_log'].hist(bins=50)

We can create a dataframe where we exclude all rows where 'rundvekt'-values >= the extreme upper limit outliers. 

In [ ]:
df_capped = df.query('Rundvekt < @extreme_upper')
df_capped

## Step 4 : Feature relationship

Now that we have explored the individual variables by themselves. We will now do a multivariate analysis.

In [ ]:
sampled_df = df_capped.sample(n=50000)

In [ ]:
ax = sns.scatterplot(x='Fartøylengde',
                     y='Rundvekt',
                     hue='Redskap FDIR',
                     data=sampled_df,
                     alpha=0.5)
ax.set_title('Fartøylengde vs Rundvekt')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))  # Plasserer legenden på siden
plt.show()

In [ ]:
features_log = ['Fartøylengde', 'Rundvekt_log', 'Varighet_log','Trekkavstand_log', 'Havdybde start']
features = ['Fartøylengde', 'Rundvekt', 'Varighet','Trekkavstand', 'Havdybde start']

In [ ]:
sns.pairplot(sampled_df, vars=features, hue='Redskap FDIR')
plt.show()

In [ ]:
sns.pairplot(sampled_df, vars=features_log, hue='Redskap FDIR')
plt.show()

We can create a correlation matrix to explore the linear association between our variables

In [ ]:
df_corr = df_capped[features].corr()
df_corr

To improve readability, we turn it into a heatmap using the Seaborn library

In [ ]:
sns.heatmap(df_corr, annot=True)

In [ ]:
df_log = df[['Art - FDIR','Rundvekt_log', 'Redskap FDIR', 'Fartøylengde', 'Havdybde start']].copy()

In [ ]:
df_log = pd.get_dummies(df_log, columns=['Art - FDIR'])
df_log = pd.get_dummies(df_log, columns=['Redskap FDIR'])

In [ ]:
df_n = df_capped[['Art - FDIR','Rundvekt', 'Redskap FDIR', 'Fartøylengde', 'Havdybde start']].copy()

In [ ]:
df_n = pd.get_dummies(df_n, columns=['Art - FDIR'])
df_n = pd.get_dummies(df_n, columns=['Redskap FDIR'])

In [ ]:
def run_model(dataframe, target):
    y = dataframe[[target]]
    X = dataframe.drop([target], axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    rf = RandomForestRegressor(random_state=42)
    
    param_grid = {
        'n_estimators': [90, 128],
        'max_depth': [12, 15],
    }

    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
    
    return grid_search.fit(X_train, y_train), X_test, y_test

In [ ]:
for (d, t) in [(df_n, 'Rundvekt'), (df_log, 'Rundvekt_log')]:
    
    model_result, X_test, y_test = run_model(d, t)
    
    print(t)
    
    best_params = model_result.best_params_
    print("Best parameters:", best_params)
    
    best_rf = model_result.best_estimator_
    
    predictions = best_rf.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    print("MSE:", mse)
    
    r2 = r2_score(y_test, predictions)
    print("R^2-score:", r2)

# KNN - regression

The code below was written in another notebook and includes more features. This is not something we aim to do in the final edition of the group exam, but it serves as an example of how we currently are exploring machine learning. 

In [ ]:
df_Knn = df_capped[['Starttidspunkt', 'Art - FDIR','Rundvekt', 'Redskap FDIR', 'Fartøylengde', 'Havdybde start', 'Hovedområde start']].copy()

Instead of months numbered 1-12, we included the fours seasons instead

In [ ]:
df_Knn['month'] = df_Knn['Starttidspunkt'].dt.month
df_Knn = df_Knn.drop(['Starttidspunkt'], axis=1)
def map_to_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df_Knn['season'] = df_Knn['month'].apply(map_to_season)

df_Knn = pd.get_dummies(df_Knn, columns=['season'])
df_Knn = df_Knn.drop(['month'], axis=1)

In [ ]:
df_Knn = pd.get_dummies(df_Knn, columns=['Art - FDIR'])
df_Knn = pd.get_dummies(df_Knn, columns=['Redskap FDIR'])
df_Knn = pd.get_dummies(df_Knn, columns=['Hovedområde start'])

In [ ]:
X = df_Knn.drop(['Rundvekt'], axis=1)
y = df_Knn[['Rundvekt']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

We can view scaling as a part of the model itself. A pipeline comes in handy for grouping the scaler and the regressor together. 
We can give the pipeline as an argument to our GridSearchCV along with the different parameters we want to optimize, our chosen scoring metrics, and the number of cross validation groups


In [ ]:
knn_pipe = Pipeline([('mms', MinMaxScaler()),
                     ('knn', KNeighborsRegressor())])

params = [{'knn__n_neighbors': [10, 20, 35, 50, 80],
         'knn__weights': ['uniform', 'distance']}]

gs_knn = GridSearchCV(knn_pipe,
                      param_grid=params,
                      scoring='neg_mean_squared_error',
                      cv=5)
gs_knn.fit(X_train, y_train)
gs_knn.best_params_

In [ ]:
predictions = gs_knn.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print("MSE: {:.2f}".format(mse))
print("r2 score: {:.2f}".format(r2))

Comparing the quality of our two different models does not entirely make sense, since Knn makes use of more features than Random Forest. We do however see that the outcome is very similar for both models that uses the capped live weight as a target variable. The R2 value indicates that none of our models perfectly explain all the variance in the dependent variable. This number did improve when using the log normalized target variable.  